In [2]:
import os

os.environ['OPENAI_API_KEY'] = ''
os.environ['HUGGINGFACEHUB_API_TOKEN'] = ''

## Basics LLMChain - Fact Extraction

In [4]:
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain
from langchain.llms import HuggingFaceHub

In [5]:
llm_hf = HuggingFaceHub(
    repo_id = "google/flan-t5-large",
    model_kwargs = {"temperature": 0, "max_length": 300}
)

/home/rebanaryal/langchain-basics/venv/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [7]:
article = '''Mount Everest is a peak in the Himalaya mountain range. It is located between Nepal and Tibet, an autonomous region of China. At 8,849 meters (29,032 feet), it is considered the tallest point on Earth. In the nineteenth century, the mountain was named after George Everest, a former Surveyor General of India. The Tibetan name is Chomolungma, which means “Mother Goddess of the World.” The Nepali name is Sagarmatha, which has various meanings.

The first ever recorded people to climb Everest were Edmund Hillary (a mountaineer from New Zealand) and his Tibetan guide Tenzing Norgay. They climbed the mountain in 1953 and hold the record together. The first records of Everest’s height came much earlier, in 1856. British surveyors recorded that Everest was the tallest peak in the world in their Great Trigonometrical Survey of the Indian subcontinent.

The Himalayan mountains have long been home to indigenous groups living in the valleys. The most famous of these are the Sherpa people. The word “Sherpa” is often used to mean mountain guide, though it actually refers to an ethnic group. The Sherpa have valuable experience in mountain climbing, which they can provide to other climbers. Most climbs of Everest would be impossible without the Sherpas’ logistical help and knowledge. However, their way of life extends beyond helping Everest climbers. Traditionally, their lifestyle has consisted of farming, herding, and trade. And, because they live at such a high altitude year round, they are accustomed to the low oxygen levels.

Climbing Mount Everest has become a popular expedition for mountain climbers. However, it is a dangerous undertaking. Climbing Everest requires a lot of experience mountaineering elsewhere, as well as a certificate of good health, equipment, and a trained Nepalese guide. The snow and ice on the mountain create deadly hazards like avalanches, and there is only a limited climbing season due to bad weather conditions. But perhaps the biggest danger is the altitude. Most climbers are not accustomed to the high altitude and low oxygen levels and rely on bottled oxygen they bring along. This is why the area above 8,000 meters (26,000 feet) elevation on Everest is called the “death zone.” Climbers who spend long periods in this region can develop altitude sickness and even brain swelling.

Mount Everest’s climbing industry has become controversial. As popularity of the climb has increased, there have been more “traffic jams” as climbers spend too much time in the death zone waiting for their chance to go to the summit. With more people has also come more pollution up near the summit as climbers often discard unwanted items all along the mountain. Additionally, the Sherpa people have been exploited by climbers, and their traditional way of life has been disrupted by foreign climbers. Sherpa guides are faced with some of the highest death rates of any field of employment, for comparatively little pay. Most disturbingly, because many climbers have died along the way, and their bodies are impossible to retrieve, climbers must frequently travel past corpses as they make their way up the mountain.'''
len(article)

3151

In [8]:
fact_extraction_prompt = PromptTemplate(
    input_variables= ["text_input"],
    template = "Extract the key facts out of this text. Don't include opinions. Give each fact a number and keep the short sentences: \n\n {text_input}")

In [11]:
fact_extraction_chain = LLMChain(llm = llm_hf, prompt = fact_extraction_prompt)
facts = fact_extraction_chain.run(article)
print(facts)

The first recorded people to climb Everest were Edmund Hillary and his Tibetan guide Tenzing Norgay. They climbed the mountain in 1953 and hold the record together. The Sherpa people have valuable experience in mountain climbing, which they can provide to other climbers.


str

## Rewrite the summart from the Fact


In [14]:
invester_update_prompt = PromptTemplate(
    input_variables = ["facts"],
    template = "extract the first climber name and the date from the paragraph. \n\n {facts}"
)

In [16]:
invester_update_chain = LLMChain(llm = llm_hf, prompt = invester_update_prompt)
invester_update = invester_update_chain.run(facts)
print(invester_update)
len(invester_update)

Edmund Hillary and his Tibetan guide Tenzing Norgay. They climbed the mountain in 1953


86

## Chaining these two chain together

In [18]:
from langchain.chains import SimpleSequentialChain, SequentialChain
full_chain = SimpleSequentialChain(chains = [fact_extraction_chain, invester_update_chain], verbose = True) #verbose when enabled True, shows the ouput stepwise
response = full_chain.run(article)



> Entering new SimpleSequentialChain chain...
The first recorded people to climb Everest were Edmund Hillary and his Tibetan guide Tenzing Norgay. They climbed the mountain in 1953 and hold the record together. The Sherpa people have valuable experience in mountain climbing, which they can provide to other climbers.
Edmund Hillary and his Tibetan guide Tenzing Norgay. They climbed the mountain in 1953

> Finished chain.
